In [103]:
# Improving the model by including various 'FEATURE IMORTANCE', based on our EDA

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

data = pd.read_csv('/Users/kauthar/Desktop/insurance.csv')

# Compute charges using linear regression models for smokers and non-smokers (against bmi and age)

charges_bmi = []
charges_age = []
for i in range(len(data)):
    if data.loc[i, 'smoker'] == 'yes':
        charges_bmi.append(1473.11 * data.loc[i, 'bmi'] - 13186.6)
        charges_age.append(305.238 * data.loc[i, 'age'] + 20294.1)
    else:
        charges_bmi.append(83.3506 * data.loc[i, 'bmi'] + 5879.42)
        charges_age.append(267.249 * data.loc[i, 'age'] - 2091.42)

data['charges_bmi'] = charges_bmi
data['charges_age'] = charges_age

# One-hot encoding for categorical features
one_hot_encoder = OneHotEncoder(sparse=False)
cat_features = ['sex', 'smoker', 'region']
one_hot_encoded = pd.DataFrame(one_hot_encoder.fit_transform(data[cat_features]))
one_hot_encoded.columns = one_hot_encoder.get_feature_names(cat_features)
data = pd.concat([data, one_hot_encoded], axis=1)
data = data.drop(cat_features, axis=1)

# Splitting the data into training and testing sets
X = data.drop('charges', axis=1)
y = data['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Set the weights for the smoker, bmi, and age variables to be higher for smokers
sample_weights = []
for i in range(len(X_train)):
    if X_train.iloc[i]['smoker_yes'] == 1:
        sample_weights.append(1.75)
    else:
        sample_weights.append(1)
    if X_train.iloc[i]['smoker_yes'] == 1:
        sample_weights[i] *= (X_train.iloc[i]['charges_bmi'] / np.mean(X_train['charges_bmi']))
        sample_weights[i] *= (X_train.iloc[i]['charges_age'] / np.mean(X_train['charges_age']))

# Training an XGBoost model on the TRAIN data
model = xgb.XGBRegressor(n_estimators=700, max_depth=3, learning_rate=0.011, random_state=42)
model.fit(X_train, y_train, sample_weight=sample_weights)

# Evaluating model on the testing data
score = model.score(X_test, y_test)
print("Model score:", score)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)


/Users/kauthar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Model score: 0.9084104495227839
RMSE: 3554.6690985856867



#### By introducing additional hyperparameters such as 'charges_bmi' and 'charges_age' under the  categorisation of boolean smoker values, we are able to develop a stronger XGBoost model (with better XGBRegressor arguments as well). The EDA we conducted gave us insights on the influence of BMI, Age and Smoker variables. Therefore, by adding "weights" appropriately to these factors and applying linear trends observed, our improved model has a Model Score of over 90% and a much lower RMSE value of 3554.67

---

In [104]:
data

,age,bmi,children,charges,charges_bmi,charges_age,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,27913.169000,26093.622,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,18,33.770,1,1725.55230,8694.169762,2719.062,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,28,33.000,3,4449.46200,8629.989800,5391.552,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33,22.705,0,21984.47061,7771.895373,6727.797,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,32,28.880,0,3866.85520,8286.585328,6460.548,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,8460.788082,11271.030,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1334,18,31.920,0,2205.98080,8539.971152,2719.062,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1335,18,36.850,0,1629.83350,8950.889610,2719.062,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1336,21,25.800,0,2007.94500,8029.865480,3520.809,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [16]:
# "BASELINE" Model (with callable function)

import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

data = pd.read_csv('/Users/kauthar/Desktop/insurance.csv')

# One-hot encoding for categorical features
one_hot_encoder = OneHotEncoder(sparse=False)
cat_features = ['sex', 'smoker', 'region']
one_hot_encoded = pd.DataFrame(one_hot_encoder.fit_transform(data[cat_features]))
one_hot_encoded.columns = one_hot_encoder.get_feature_names(cat_features)
data = pd.concat([data, one_hot_encoded], axis=1)
data = data.drop(cat_features, axis=1)

# Training an XGBoost model on the entire dataset
model = xgb.XGBRegressormodel = xgb.XGBRegressor(n_estimators=1000, max_depth=4, learning_rate=0.05, random_state=42)
model.fit(data.drop('charges', axis=1), data['charges'])

def predict_charges(age, sex, bmi, children, smoker, region):
    # Converting the input values into a pandas dataframe
    input_df = pd.DataFrame({'age': [age], 'sex': [sex], 'bmi': [bmi], 'children': [children], 'smoker': [smoker], 'region': [region]})

    # One-hot encoding: for categorical features
    one_hot_encoded = pd.DataFrame(one_hot_encoder.transform(input_df[cat_features]))
    one_hot_encoded.columns = one_hot_encoder.get_feature_names(cat_features)
    input_df = pd.concat([input_df, one_hot_encoded], axis=1)
    input_df = input_df.drop(cat_features, axis=1)

    # Use TRAINED XGBoost model to predict the charges
    prediction = model.predict(input_df)[0]
    return prediction


/Users/kauthar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [107]:
# Improved Model (with callable function)

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

data = pd.read_csv('/Users/kauthar/Desktop/insurance.csv')

# Compute charges using linear regression models for smokers and non-smokers (against bmi and age)
charges_bmi = []
charges_age = []
for i in range(len(data)):
    if data.loc[i, 'smoker'] == 'yes':
        charges_bmi.append(1473.11 * data.loc[i, 'bmi'] - 13186.6)
        charges_age.append(305.238 * data.loc[i, 'age'] + 20294.1)
    else:
        charges_bmi.append(83.3506 * data.loc[i, 'bmi'] + 5879.42)
        charges_age.append(267.249 * data.loc[i, 'age'] - 2091.42)

data['charges_bmi'] = charges_bmi
data['charges_age'] = charges_age

# One-hot encoding for categorical features
one_hot_encoder = OneHotEncoder(sparse=False)
cat_features = ['sex', 'smoker', 'region']
one_hot_encoded = pd.DataFrame(one_hot_encoder.fit_transform(data[cat_features]))
one_hot_encoded.columns = one_hot_encoder.get_feature_names(cat_features)
data = pd.concat([data, one_hot_encoded], axis=1)
data = data.drop(cat_features, axis=1)

# Splitting the data into training and testing sets
X = data.drop('charges', axis=1)
y = data['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the weights for the smoker, bmi, and age variables to be higher for smokers
sample_weights = []
for i in range(len(X_train)):
    if X_train.iloc[i]['smoker_yes'] == 1:
        sample_weights.append(0.75)
    else:
        sample_weights.append(1)
    if X_train.iloc[i]['smoker_yes'] == 1:
        sample_weights[i] *= (X_train.iloc[i]['charges_bmi'] / np.mean(X_train['charges_bmi']))
        sample_weights[i] *= (X_train.iloc[i]['charges_age'] / np.mean(X_train['charges_age']))

# Training an XGBoost model on the entire dataset
model = xgb.XGBRegressor(n_estimators=700, max_depth=3, learning_rate=0.011, random_state=42)
model.fit(data.drop('charges', axis=1), data['charges'])

def IMPROVED_predict_charges(age, sex, bmi, children, smoker, region):
    # Compute charges using linear regression models for smokers and non-smokers (against bmi and age)
    if smoker == 'yes':
        charges_bmi = 1473.11 * bmi - 13186.6
        charges_age = 305.238 * age + 20294.1
    else:
        charges_bmi = 83.3506 * bmi + 5879.42
        charges_age = 267.249 * age - 2091.42

    # Converting the input values into a pandas dataframe
    input_df = pd.DataFrame({'age': [age], 'sex': [sex], 'bmi': [bmi], 'children': [children], 'smoker': [smoker], 'region': [region], 'charges_bmi': [charges_bmi], 'charges_age': [charges_age]})

    # One-hot encoding for categorical features
    one_hot_encoded = pd.DataFrame(one_hot_encoder.transform(input_df[cat_features]))
    one_hot_encoded.columns = one_hot_encoder.get_feature_names(cat_features)
    input_df = pd.concat([input_df, one_hot_encoded], axis=1)
    input_df = input_df.drop(cat_features, axis=1)

    # Use TRAINED XGBoost model to predict the charges
    prediction = model.predict(input_df)[0]
    return prediction


/Users/kauthar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



### Tool for Insurance Analysts
---


In [108]:
import ipywidgets as widgets
from IPython.display import display

# Create widgets for input
age_widget = widgets.IntSlider(description="Age:", min=18, max=100, value=30, style={'description_width': 'initial'})
sex_widget = widgets.Dropdown(options=[('Male', 'male'), ('Female', 'female')], description="Sex:", style={'description_width': 'initial'})
bmi_widget = widgets.FloatText(description="BMI:", min=10, max=50, value=20, style={'description_width': 'initial'})
children_widget = widgets.IntSlider(description="Children:", min=0, max=10, value=0, style={'description_width': 'initial'})
smoker_widget = widgets.Dropdown(options=[('No', 'no'), ('Yes', 'yes')], description="Smoker:", style={'description_width': 'initial'})
region_widget = widgets.Dropdown(options=[('Northeast', 'northeast'), ('Northwest', 'northwest'), ('Southeast', 'southeast'), ('Southwest', 'southwest')], description="Region:", style={'description_width': 'initial'})

# Define a function to handle user input
def handle_submit(sender):
    # Get the current values of widgets
    age_value = age_widget.value
    sex_value = sex_widget.value
    bmi_value = bmi_widget.value
    children_value = children_widget.value
    smoker_value = smoker_widget.value
    region_value = region_widget.value
    print(str(age_value)+" year old "+sex_value+" with "+str(children_value)+" child(ren), living in the "+region_value)
    print("BMI -> "+str(bmi_value))
    print("Smoker? "+smoker_value)
    
    # Call the predict_charges function with user input
    IMPROVED_predicted_charges = IMPROVED_predict_charges(age_value, sex_value, bmi_value, children_value, smoker_value, region_value)

    # Display the predicted charges
    print('Predicted charges: $', round(IMPROVED_predicted_charges, 2))

# Define a button to submit user input
submit_button = widgets.Button(description='Predict Charges', button_style='success')
submit_button.on_click(handle_submit)

# Display the input widgets and submit button
display(age_widget, sex_widget, bmi_widget, children_widget, smoker_widget, region_widget, submit_button)


IntSlider(value=30, description='Age:', min=18, style=SliderStyle(description_width='initial'))

Dropdown(description='Sex:', options=(('Male', 'male'), ('Female', 'female')), style=DescriptionStyle(descript…

FloatText(value=20.0, description='BMI:', style=DescriptionStyle(description_width='initial'))

IntSlider(value=0, description='Children:', max=10, style=SliderStyle(description_width='initial'))

Dropdown(description='Smoker:', options=(('No', 'no'), ('Yes', 'yes')), style=DescriptionStyle(description_wid…

Dropdown(description='Region:', options=(('Northeast', 'northeast'), ('Northwest', 'northwest'), ('Southeast',…

Button(button_style='success', description='Predict Charges', style=ButtonStyle())

User Test Cases (index) : 100